In [37]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2
from ppca import PPCA

data = pd.read_csv('mentalhealthanalysis/adult21.csv')

print(data.info())
data.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29482 entries, 0 to 29481
Columns: 622 entries, URBRRL to POVRATTC_A
dtypes: float64(437), int64(184), object(1)
memory usage: 139.9+ MB
None


,URBRRL,RATCAT_A,IMPINCFLG_A,CVDVAC2YR_A,CVDVAC2MR_A,CVDVAC1YR_A,CVDVAC1MR_A,SHTCVD19AV_A,SHTCVD19NM_A,SHTCVD19_A,...,PROXYREL_A,PROXY_A,AVAIL_A,HHSTAT_A,INTV_MON,RECTYPE,IMPNUM_A,WTFA_A,HHX,POVRATTC_A
29477,4,14,1,2021.0,1.0,2021.0,6.0,NaN,3.0,1.0,...,NaN,NaN,1,1,12,10,1,3203.634,H012375,5.11
29478,4,10,0,2021.0,3.0,2021.0,4.0,NaN,2.0,1.0,...,NaN,NaN,1,1,12,10,1,1556.049,H052160,3.03
29479,4,8,0,2021.0,4.0,2021.0,11.0,NaN,3.0,1.0,...,NaN,NaN,1,1,12,10,1,6238.789,H051563,2.07
29480,4,8,0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,1,1,12,10,1,4769.800,H058432,2.05
29481,4,13,0,2021.0,4.0,2021.0,5.0,NaN,2.0,1.0,...,NaN,NaN,1,1,12,10,1,6336.260,H052354,4.89


In [38]:
#Drop all columns with <1/3 Response Rate
dfDrop = data.dropna(axis=1, thresh=10000)
dfDrop.info()

# TODO Find list of singular columns that will have no bearing
 # Maybe find list of groups of columns that are very related?

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29482 entries, 0 to 29481
Columns: 315 entries, URBRRL to POVRATTC_A
dtypes: float64(130), int64(184), object(1)
memory usage: 70.9+ MB


In [63]:
# Check out correlation between age of diagnosis and mental health (starting with cancer)

# Pairs of cancers and age of diagnosis (could make tuples later)
cols = ['BLADDCAN_A', 'BLADDAGETC_A', 'BLOODCAN_A', 'BLOODAGETC_A', 'BONECAN_A', 'BONEAGETC_A', 'BRAINCAN_A', 'BRAINAGETC_A', 'BREASCAN_A', 
        'BREASAGETC_A', 'CERVICAN_A', 'CERVIAGETC_A', 'COLONCAN_A', 'COLONAGETC_A', 'ESOPHCAN_A', 'ESOPHAGETC_A', 'GALLBCAN_A', 'GALLBAGETC_A',
        'LARYNCAN_A', 'LARYNAGETC_A', 'LEUKECAN_A', 'LEUKEAGETC_A', 'LIVERCAN_A', 'LIVERAGETC_A', 'LUNGCAN_A', 'LUNGAGETC_A', 'LYMPHCAN_A',
        'LYMPHAGETC_A', 'MELANCAN_A', 'MELANAGETC_A', 'MOUTHCAN_A', 'MOUTHAGETC_A', 'OVARYCAN_A', 'OVARYAGETC_A', 'PANCRCAN_A', 'PANCRAGETC_A',
        'PROSTCAN_A', 'PROSTAGETC_A', 'RECTUCAN_A', 'RECTUAGETC_A', 'SKNMCAN_A', 'SKNMAGETC_A', 'SKNNMCAN_A', 'SKNNMAGETC_A', 'SKNDKCAN_A', 
        'SKNDKAGETC_A', 'STOMACAN_A', 'STOMAAGETC_A', 'THROACAN_A', 'THROAAGETC_A', 'THYROCAN_A', 'THYROAGETC_A', 'UTERUCAN_A', 'UTERUAGETC_A',
        'HDNCKCAN_A', 'HDNCKAGETC_A', 'COLRCCAN_A', 'COLRCAGETC_A', 'OTHERCANP_A', 'OTHERAGETC_A']

# Separate age of diagnosis
age_cols = []
for i in range(len(cols)):
    if(i % 2 == 1):
        age_cols.append(cols[i])
print(age_cols)

df_can = data[cols].copy()
df_can.info()

# Get means 
df_can_age = data[age_cols].copy()
df_can_age.mean()

['BLADDAGETC_A', 'BLOODAGETC_A', 'BONEAGETC_A', 'BRAINAGETC_A', 'BREASAGETC_A', 'CERVIAGETC_A', 'COLONAGETC_A', 'ESOPHAGETC_A', 'GALLBAGETC_A', 'LARYNAGETC_A', 'LEUKEAGETC_A', 'LIVERAGETC_A', 'LUNGAGETC_A', 'LYMPHAGETC_A', 'MELANAGETC_A', 'MOUTHAGETC_A', 'OVARYAGETC_A', 'PANCRAGETC_A', 'PROSTAGETC_A', 'RECTUAGETC_A', 'SKNMAGETC_A', 'SKNNMAGETC_A', 'SKNDKAGETC_A', 'STOMAAGETC_A', 'THROAAGETC_A', 'THYROAGETC_A', 'UTERUAGETC_A', 'HDNCKAGETC_A', 'COLRCAGETC_A', 'OTHERAGETC_A']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29482 entries, 0 to 29481
Data columns (total 60 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   BLADDCAN_A    3654 non-null   float64
 1   BLADDAGETC_A  121 non-null    float64
 2   BLOODCAN_A    3654 non-null   float64
 3   BLOODAGETC_A  25 non-null     float64
 4   BONECAN_A     3654 non-null   float64
 5   BONEAGETC_A   30 non-null     float64
 6   BRAINCAN_A    3654 non-null   float64
 7   BRAINAGETC_A  23 non-

BLADDAGETC_A    64.322314
BLOODAGETC_A    56.920000
BONEAGETC_A     56.933333
BRAINAGETC_A    40.913043
BREASAGETC_A    56.944126
CERVIAGETC_A    33.475000
COLONAGETC_A    61.227027
ESOPHAGETC_A    64.909091
GALLBAGETC_A    61.800000
LARYNAGETC_A    68.750000
LEUKEAGETC_A    49.629630
LIVERAGETC_A    63.761905
LUNGAGETC_A     66.756522
LYMPHAGETC_A    51.573770
MELANAGETC_A    56.264550
MOUTHAGETC_A    57.928571
OVARYAGETC_A    45.762500
PANCRAGETC_A    63.000000
PROSTAGETC_A    64.946067
RECTUAGETC_A    57.875000
SKNMAGETC_A     57.570122
SKNNMAGETC_A    56.821293
SKNDKAGETC_A    61.271795
STOMAAGETC_A    64.315789
THROAAGETC_A    60.548387
THYROAGETC_A    44.680672
UTERUAGETC_A    49.017094
HDNCKAGETC_A    59.979167
COLRCAGETC_A    61.282828
OTHERAGETC_A    55.600000
dtype: float64

1


In [64]:
# pca = PCA(n_components=315) # no nans allowed
# pca.fit(df_can)

ppca = PPCA()
ppca.fit(data=df_can_age.to_numpy(), d=60, verbose=True)

# X, y = load_digits(return_X_y=True)
# kbest = SelectKBest(chi2, k=20).fit_transform(df_can)

1.0
0.24960768566249092
0.0006766826685473681
1.2198294217502514e-05
1.2190755578678214e-05
1.2183355916906002e-05
1.2175921876744766e-05


In [5]:
# Vars
countColNan = data.isnull().sum(axis=0).tolist() # Column NaN counts
numRows = len(data) # Number of rows
colNames = data.columns

percList = [((x / numRows)*100) for x in countColNan] # Percent Missing Data

addBool = [ind < 1 for ind in percList] 

addList = []

for i in range(len(addBool)):
    if addBool[i]:
        addList.append(colNames[i])

dfNew = data[addList].copy()
dfNew = dfNew.drop(['HHX'], axis=1)
dfNew.dropna()

goodCols = ['EDUCP_A', 'PHSTAT_A','LSATIS11R_A', 'LSATIS4R_A', 'ANXEV_A', 'DEPEV_A','WEIGHTLBTC_A','NOTCOV_A']#'WELLVIS_A']

dfPCA = dfNew[goodCols]